In [1]:
#multi-agent testrun using pettingzoo
#still need to update the actionset
#still need to update reward function
#trial to see if it works

#update the actions so that it selects the frequency range of a particular radar first, and then picks a sub-band from inside it
#fix the frequency hopping of the radar so that it only transmits one frequency per radar per step.

#after finalizing actions and rewards, test on vanilla dqn and sstart implementing ddqn with per

#fixed the radar frquency issue, need to fix the states, obs, info etc tuple being returned by env to the dqn
#after that, train dqn
#finally move to ddqn with per

In [2]:
import numpy as np
import math
import gymnasium as gym
from gym import Env
from gym import spaces
import random
import numpy as np
from IPython.display import clear_output
import os
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [3]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)]]

interval = (freq_agile_radar_versions[0][1] - freq_agile_radar_versions[0][0])/20
print (interval)

hop_states = []
for i in range (0,20):
    
    hop_states.append(int(freq_agile_radar_versions[0][0]+ interval * i))

print (hop_states)
print (len(hop_states))


20000000.0
[9000000000, 9020000000, 9040000000, 9060000000, 9080000000, 9100000000, 9120000000, 9140000000, 9160000000, 9180000000, 9200000000, 9220000000, 9240000000, 9260000000, 9280000000, 9300000000, 9320000000, 9340000000, 9360000000, 9380000000]
20


In [4]:
#hop_states = np.linspace(9e9, 9.4e9, 20e6)
jamming_bandwidths = [10e6, 20e6, 30e6]

In [5]:
from pettingzoo import ParallelEnv
from gym import spaces
import numpy as np

class ParallelRadarJammingEnv(ParallelEnv):
    def __init__(self, hop_states, jamming_bandwidths, max_hop_length=8):
        super().__init__()

        self.hop_states = hop_states
        self.low = self.hop_states[0]
        self.interval = self.hop_states[1] - self.hop_states[0]
        self.n_frequencies = len(hop_states)
        self.max_hop_length = max_hop_length

        # agent params, action spaces and observation spaces
        self.agents = ["jammer_0", "jammer_1", "jammer_2", "jammer_3", "jammer_4", "jammer_5"]
        self.possible_agents = self.agents[:]
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.agents)}

        self.jamming_bandwidths = jamming_bandwidths
        self.n_bandwidths = len(self.jamming_bandwidths)
        self.action_space = {agent: spaces.Discrete(self.max_hop_length * self.n_bandwidths) for agent in self.agents}

        self.observation_spaces = {
            agent: spaces.Box(low=0, high=1, shape=(self.n_frequencies,), dtype=np.float32) for agent in self.agents
        }
        print(self.observation_spaces)
        # step and reward vars
        self.hopping_patterns = None
        self.current_frequencies = None
        self.current_steps = None
        self.cumulative_rewards = None
        self.net_agent_rewards = None
        self.terminations = None
        self.rewards = None
        self.info = None

    def index_to_action(self, index):
        frequency_index = index // self.n_bandwidths
        bandwidth_index = index % self.n_bandwidths
        frequency = int(self.low + frequency_index * self.interval)
        bandwidth = self.jamming_bandwidths[bandwidth_index]
        return frequency, bandwidth


    def generate_hopping_pattern(self):
        return np.random.choice(self.hop_states, self.max_hop_length, replace=False)

    def next_frequency(self, agent):
        return self.hopping_patterns[agent][self.current_steps[agent] % self.max_hop_length]

    def reset(self):

        #set radar freq
        self.radar_frequency = self.generate_hopping_pattern()[0]

        #init states for agents
        self.current_steps = {agent: 0 for agent in self.agents}
        self.cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.rewards = {agent: 0 for agent in self.agents}
        self.info = {agent: {} for agent in self.agents}

        #obs for agents
        observations = {agent: self.observe(agent) for agent in self.agents}
        return observations


    def observe(self, agent):

        #one-hot encoded obs of the radar freqs used
        observation = np.zeros(self.n_frequencies)
        freq_index = np.where(self.hop_states == self.radar_frequency)[0][0]
        observation[freq_index] = 1
        return observation


    def step(self, actions):
        total_jammed_frequencies = 0
        self.net_agent_rewards = 0

        for agent, action in actions.items():

            frequency, bandwidth = self.index_to_action(action)

            lower_bound = frequency - bandwidth
            upper_bound = frequency + bandwidth

            radar_frequency = self.radar_frequency

            if lower_bound <= radar_frequency <= upper_bound:
                reward = 100
                total_jammed_frequencies += 1
            else:
                reward = -50

            if 2 * (bandwidth / self.interval) > 5:
                reward -= 1
            else:
                reward += 0

            jam_threshold = len(self.hop_states) / len(self.agents)
            if total_jammed_frequencies > jam_threshold:
                reward += 5
            else:
                reward -= 5

            #update rewards
            self.current_steps[agent] += 1
            self.cumulative_rewards[agent] += reward
            self.net_agent_rewards += reward

            #num steps per episode
            if self.current_steps[agent] >= 100:
                self.terminations[agent] = True

            self.rewards[agent] = reward

        #update radar req for next step
        self.radar_frequency = self.generate_hopping_pattern()[0]

        #return the observations, rewards, done status, and info for each agent
        observations = {agent: self.observe(agent) for agent in self.agents}
        rewards = {agent: self.rewards[agent] for agent in self.agents}
        done = {agent: self.terminations[agent] for agent in self.agents}
        info = {agent: self.info[agent] for agent in self.agents}

        return observations, rewards, done, info

    def render(self):
        for agent in self.agents:
            print(f"Agent {agent}, Step: {self.current_steps[agent]}, Radar Frequency: {self.radar_frequency}")


In [6]:
env = ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

observations = env.reset()
done = {agent: False for agent in env.agents}
cumulative_reward = {agent: 0 for agent in env.agents}

while not all(done.values()):

    actions = {agent: env.action_space[agent].sample() for agent in env.agents}
    observations, rewards, done, infos = env.step(actions)

    for agent in env.agents:
        cumulative_reward[agent] += rewards[agent]

    env.render()
    print(f"Actions: {actions}, Rewards: {rewards}, Cumulative Rewards: {cumulative_reward}")
    print('\n')


{'jammer_0': Box(0.0, 1.0, (20,), float32), 'jammer_1': Box(0.0, 1.0, (20,), float32), 'jammer_2': Box(0.0, 1.0, (20,), float32), 'jammer_3': Box(0.0, 1.0, (20,), float32), 'jammer_4': Box(0.0, 1.0, (20,), float32), 'jammer_5': Box(0.0, 1.0, (20,), float32)}
Agent jammer_0, Step: 1, Radar Frequency: 9300000000
Agent jammer_1, Step: 1, Radar Frequency: 9300000000
Agent jammer_2, Step: 1, Radar Frequency: 9300000000
Agent jammer_3, Step: 1, Radar Frequency: 9300000000
Agent jammer_4, Step: 1, Radar Frequency: 9300000000
Agent jammer_5, Step: 1, Radar Frequency: 9300000000
Actions: {'jammer_0': 2, 'jammer_1': 18, 'jammer_2': 16, 'jammer_3': 2, 'jammer_4': 8, 'jammer_5': 6}, Rewards: {'jammer_0': -55, 'jammer_1': -55, 'jammer_2': -55, 'jammer_3': -55, 'jammer_4': -55, 'jammer_5': -55}, Cumulative Rewards: {'jammer_0': -55, 'jammer_1': -55, 'jammer_2': -55, 'jammer_3': -55, 'jammer_4': -55, 'jammer_5': -55}


Agent jammer_0, Step: 2, Radar Frequency: 9020000000
Agent jammer_1, Step: 2, Rada

In [7]:
#setting up replay memory

Transition = namedtuple('Transition',
('state', 'action', 'next_state', 'reward')
)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

# Neural network for the DDQN
class DQN(nn.Module):
    def __init__(self, obs_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(obs_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [9]:
class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.buffer = []
        self.priorities = np.zeros((capacity,), dtype=np.float32)
        self.capacity = capacity
        self.pos = 0
        self.alpha = alpha

    def add(self, state, action, reward, next_state, done):
        max_priority = self.priorities.max() if self.buffer else 1.0
        if len(self.buffer) < self.capacity:
            self.buffer.append((state, action, reward, next_state, done))
        else:
            self.buffer[self.pos] = (state, action, reward, next_state, done)

        self.priorities[self.pos] = max_priority  # Assign max priority to new transitions
        self.pos = (self.pos + 1) % self.capacity

    def sample(self, batch_size, beta=0.4):
        if len(self.buffer) == self.capacity:
            priorities = self.priorities
        else:
            priorities = self.priorities[:self.pos]

        probabilities = priorities ** self.alpha
        probabilities /= probabilities.sum()

        indices = np.random.choice(len(self.buffer), batch_size, p=probabilities)
        samples = [self.buffer[idx] for idx in indices]

        total = len(self.buffer)
        weights = (total * probabilities[indices]) ** (-beta)
        weights /= weights.max()  # Normalize for stability

        states, actions, rewards, next_states, dones = zip(*samples)
        return (np.array(states), np.array(actions), np.array(rewards),
                np.array(next_states), np.array(dones), indices, np.array(weights))

    def update_priorities(self, batch_indices, batch_priorities):
        for idx, priority in zip(batch_indices, batch_priorities):
            self.priorities[idx] = priority

    def size(self):
        return len(self.buffer)

In [10]:
def update_ddqn(shared_dqn, target_dqn, optimizer, replay_buffer, batch_size, gamma, beta):
    if replay_buffer.size() < batch_size:
        return
    
    # Sample from prioritized replay buffer
    states, actions, rewards, next_states, dones, indices, weights = replay_buffer.sample(batch_size, beta)
    
    states = torch.FloatTensor(states)
    actions = torch.LongTensor(actions).unsqueeze(1)
    rewards = torch.FloatTensor(rewards).unsqueeze(1)
    next_states = torch.FloatTensor(next_states)
    dones = torch.FloatTensor(dones).unsqueeze(1)
    weights = torch.FloatTensor(weights).unsqueeze(1)

    # Current Q-values
    q_values = shared_dqn(states).gather(1, actions)

    # Double DQN update rule
    next_actions = shared_dqn(next_states).argmax(1, keepdim=True)
    next_q_values = target_dqn(next_states).gather(1, next_actions)

    target_q_values = rewards + gamma * next_q_values * (1 - dones)

    td_errors = target_q_values - q_values
    loss = (weights * td_errors ** 2).mean()  # Weighted MSE loss with importance sampling correction

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Update priorities based on new TD errors
    new_priorities = td_errors.abs().detach().numpy() + 1e-5
    replay_buffer.update_priorities(indices, new_priorities)


In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt

# Function to select actions
def select_action(state, epsilon, action_size, model):
    if np.random.rand() < epsilon:  # Explore
        return np.random.randint(action_size)  # Random action
    else:  # Exploit
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convert to tensor
            q_values = model(state_tensor)  # Get Q-values
            return q_values.argmax().item()  # Best action

# Update target function
def update_target(main_dqn, target_dqn):
    target_dqn.load_state_dict(main_dqn.state_dict())

# Function to plot rewards
'''def plot_rewards(rewards, mean_rewards):

    plt.close('all')
    plt.plot(rewards, color='blue', label='Total Rewards')
    plt.plot(mean_rewards, color='orange', label='Mean Rewards (every 100 episodes)')    
    plt.xlim(0, len(rewards))
    plt.ylim(min(rewards), max(rewards) + 10)
    plt.title('Total Rewards Over Episodes')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.legend()
    plt.grid(True)
    plt.pause(0.1)  # Pause to allow the plot to update'''

# Training function
def train_ddqn_per(env, num_episodes, batch_size, gamma=0.99, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995, 
                   beta_start=0.4, beta_end=1.0, beta_increment_per_episode=0.001, target_update_freq=10):

    agent_id = env.agents[0]

    if callable(env.observation_space):
        obs_size = env.observation_space(agent_id).shape[0]
    else:
        obs_size = env.observation_space[agent_id].shape[0]

    if callable(env.action_space):
        action_size = env.action_space(agent_id).n
    else:
        action_size = env.action_space[agent_id].n

    # Initialize shared DQN and target DQN
    shared_dqn = DQN(obs_size, action_size)
    target_dqn = DQN(obs_size, action_size)
    target_dqn.load_state_dict(shared_dqn.state_dict())

    # Optimizer and replay buffer setup
    optimizer = optim.Adam(shared_dqn.parameters(), lr=1e-3)  # Learning rate set to 0.001
    replay_buffer = PrioritizedReplayBuffer(10000)

    epsilon = epsilon_start
    beta = beta_start

    total_rewards = []  # List to store total rewards per episode
    mean_rewards = []   # List to store mean rewards

    plt.ion()  # Turn on interactive mode
    plt.figure(figsize=(10, 5))

    for episode in range(num_episodes):
        states = env.reset()  # Reset environment
        episode_rewards = {agent: 0 for agent in env.agents}
        done = {agent: False for agent in env.agents}

        while not all(done.values()):
            actions = {}
            for agent in env.agents:
                if not done[agent]:
                    actions[agent] = select_action(states[agent], epsilon, action_size, shared_dqn)

            # Step the environment
            next_states, rewards, dones, _ = env.step(actions)

            # Add experiences to replay buffer
            for agent in env.agents:
                replay_buffer.add(states[agent], actions[agent], rewards[agent], next_states[agent], dones[agent])
                episode_rewards[agent] += rewards[agent]

            # Update DDQN
            update_ddqn(shared_dqn, target_dqn, optimizer, replay_buffer, batch_size, gamma, beta)

            states = next_states
            done = dones

        total_reward = sum(episode_rewards.values())
        total_rewards.append(total_reward)  # Append total reward for the episode

        # Calculate mean rewards for every 100 episodes
        if (episode + 1) % 100 == 0:
            mean_reward = np.mean(total_rewards[-100:])
            mean_rewards.append(mean_reward)
        else:
            mean_rewards.append(mean_rewards[-1] if mean_rewards else 0)  # Fill with last mean if not enough episodes yet

        # Update the plot
        #plot_rewards(total_rewards, mean_rewards)

        # Decay epsilon and update beta
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        beta = min(beta_end, beta + beta_increment_per_episode)

        # Periodically update the target network
        if episode % target_update_freq == 0:
            update_target(shared_dqn, target_dqn)

        print(f'Episode {episode + 1}: Total reward = {total_reward}')

    plt.ioff()  # Turn off interactive mode
    plt.show()  # Show the final plot

    return shared_dqn


In [12]:
# BATCH_SIZE = 128
# GAMMA = 0.99
# EPS_START = 0.9
# EPS_END = 0.05
# EPS_DECAY = 1000
# TAU = 0.005
# LR = 1e-4

env = ParallelRadarJammingEnv(hop_states, jamming_bandwidths)
train_ddqn_per(env, num_episodes=10000, 
                batch_size=64,  # Increased batch size
                gamma=0.95, 
                epsilon_start=0.99, 
                epsilon_end=0.05, 
                epsilon_decay=0.98,  # Slower decay
                beta_start=0.2,  # Higher initial beta
                beta_end=1.0, 
                beta_increment_per_episode=0.0001, 
                target_update_freq=20)  # More frequent target updates


{'jammer_0': Box(0.0, 1.0, (20,), float32), 'jammer_1': Box(0.0, 1.0, (20,), float32), 'jammer_2': Box(0.0, 1.0, (20,), float32), 'jammer_3': Box(0.0, 1.0, (20,), float32), 'jammer_4': Box(0.0, 1.0, (20,), float32), 'jammer_5': Box(0.0, 1.0, (20,), float32)}


/home/abhimanyu/miniconda3/envs/rlenv/lib/python3.10/site-packages/pettingzoo/utils/env.py:358: UserWarning: Your environment should override the observation_space function. Attempting to use the observation_spaces dict attribute.
  warnings.warn(
/tmp/ipykernel_18356/2551386287.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.priorities[idx] = priority


Episode 1: Total reward = -25340
Episode 2: Total reward = -21540
Episode 3: Total reward = -24300
Episode 4: Total reward = -22030
Episode 5: Total reward = -19710
Episode 6: Total reward = -18190
Episode 7: Total reward = -20660
Episode 8: Total reward = -20120
Episode 9: Total reward = -16190
Episode 10: Total reward = -14560
Episode 11: Total reward = -18470
Episode 12: Total reward = -17340
Episode 13: Total reward = -16740
Episode 14: Total reward = -13410
Episode 15: Total reward = -15870
Episode 16: Total reward = -17130
Episode 17: Total reward = -14380
Episode 18: Total reward = -15580
Episode 19: Total reward = -8360
Episode 20: Total reward = -13550
Episode 21: Total reward = -15950
Episode 22: Total reward = -6440
Episode 23: Total reward = -8060
Episode 24: Total reward = -11870
Episode 25: Total reward = -11710
Episode 26: Total reward = -10260
Episode 27: Total reward = -5250
Episode 28: Total reward = -14260
Episode 29: Total reward = -11510
Episode 30: Total reward = 

<Figure size 1000x500 with 0 Axes>

DQN(
  (fc1): Linear(in_features=20, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=24, bias=True)
)